In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

# read 데이터
dly = pd.read_csv('AFSNT_DLY.CSV', encoding="cp949")
dly.head()


,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,2019,9,16,월,ARP1,ARP3,L,L1702,A,9:05,NaN,NaN
1,2019,9,16,월,ARP3,ARP1,L,L1702,D,7:55,NaN,NaN
2,2019,9,16,월,`,ARP3,L,L1720,A,14:40,NaN,NaN
3,2019,9,16,월,ARP3,ARP1,L,L1720,D,13:30,NaN,NaN
4,2019,9,16,월,ARP4,ARP3,L,L1808,A,20:10,NaN,NaN


In [2]:
# 필요없는 column 제거
dly.drop(columns=['FLT', 'DLY', 'DLY_RATE'], axis=1, inplace=True)

In [3]:
dly.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,AOD,STT
0,2019,9,16,월,ARP1,ARP3,L,A,9:05
1,2019,9,16,월,ARP3,ARP1,L,D,7:55
2,2019,9,16,월,`,ARP3,L,A,14:40
3,2019,9,16,월,ARP3,ARP1,L,D,13:30
4,2019,9,16,월,ARP4,ARP3,L,A,20:10


In [4]:
# 파생변수
dly['ARPODP'] = dly['ARP'] + '_' + dly['ODP']

In [5]:
dly.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,AOD,STT,ARPODP
0,2019,9,16,월,ARP1,ARP3,L,A,9:05,ARP1_ARP3
1,2019,9,16,월,ARP3,ARP1,L,D,7:55,ARP3_ARP1
2,2019,9,16,월,`,ARP3,L,A,14:40,`_ARP3
3,2019,9,16,월,ARP3,ARP1,L,D,13:30,ARP3_ARP1
4,2019,9,16,월,ARP4,ARP3,L,A,20:10,ARP4_ARP3


In [4]:
# 분 삭제
dly['STT'] = pd.to_datetime(dly['STT'],format= '%H:%M').dt.hour

In [5]:
import urllib
import json
import pandas as pd
import dateutil.parser
#날씨데이터 json 읽어오는 함수

#나중에 9월 1일을 9월 16일로 바꾼 후 실행..
#(읽어오는거 확인용으로 9월 1일 부터...)
def readJSON(area):
    
    request = urllib.request.urlopen('https://api.aerisapi.com/forecasts/'+area+',korea?from=09/16/2019&format=json&filter=1hr&limit=999&client_id=gHOhinKWCL1fwDUpI1Ec7&client_secret=TRamBTyXpORXcmDKVqc2S4i4mnCjVxxHMt6cllui')
    response = request.read()
    data= json.loads(response)


    if data['success']:
        return data

    else:
        print("An error occurred: %s" % (data['error']['description']))
        return ""
        request.close()

In [32]:
#날씨 csv 저장
def storeWeather():
    weather=pd.DataFrame(columns=['SDT_YY', 'SDT_MM', 'SDT_DD','STT','ARP','temp', 'hum', 'dew', 'windSpeed','hpa'])

    for i in range(15):
        area=["seoul","busan","jeju","daegu","ulsan","cheongju","muan","gwangju","yeosu"
              ,"yangyang","pohang","sacheon","gunsan","wonju","incheon"]
        d=readJSON(area[i])
        response=d['response'][0]
        data=response['periods']
    
    
        for j in range(len(data)):
            temp=data[j]
            date=dateutil.parser.parse(temp['dateTimeISO'])
            temp2=pd.Series([date.year,date.month,date.day,date.hour,"ARP"+str(i+1),temp['tempC'],
                         temp['humidity'],temp['dewpointC'],temp['windGustKTS'],temp['pressureMB']],
                        index=['SDT_YY', 'SDT_MM', 'SDT_DD','STT','ARP','temp', 'hum', 'dew', 'windSpeed','hpa'])
        
            weather=weather.append(temp2,ignore_index=True)
    
    #fog 모델하고의 단위 맞추는 작업
    weather['windSpeed']=weather['windSpeed'].astype("float")*1852/3600
    
    weather.to_csv("newWeather.csv",index=False)

In [33]:
# csv 파일 저장하고 싶을때만 실행..
storeWeather() 

# 날씨데이터 합치기---------------------

we=pd.read_csv('newWeather.csv', encoding="cp949")

df= pd.merge(we, dly, on=['SDT_YY', 'SDT_MM', 'SDT_DD', 'ARP','STT'])
df

#---------------------------------------

,SDT_YY,SDT_MM,SDT_DD,STT,ARP,temp,hum,dew,windSpeed,hpa,SDT_DY,ODP,FLO,AOD
0,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,L,D
1,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP12,J,D
2,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,F,D
3,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,F,D
4,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,J,D
5,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,A,D
6,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,H,D
7,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,F,D
8,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,A,D
9,2019,9,16,6,ARP1,23,93,21,1.543333,1016,월,ARP3,H,D


In [13]:
import warnings
warnings.filterwarnings(action="ignore")

for i in range(6):
    update=[]
    for j in range(24):
        newTemp=0.0
        newHum=0.0
        newDew=0.0
        newWindSpeed=0.0
        newHpa=0.0
        newDew=0.0
        for k in range(5):
            newTemp += df[(df['SDT_DD']==25+i-k+1) & (df['STT']==j)]['temp']
            newHum += df[(df['SDT_DD']==25+i-k+1) & (df['STT']==j)]['hum']
            newDew += df[(df['SDT_DD']==25+i-k+1) & (df['STT']==j)]['dew']
            newWindSpeed += df[(df['SDT_DD']==25+i-k+1) & (df['STT']==j)]['windSpeed']
            newHpa += df[(df['SDT_DD']==25+i-k+1) & (df['STT']==j)]['hpa']
            #newDew += df[(df['SDT_DD']==25+i-k+1) & (df['STT']==j)]['dew']
#            print(df[(df['SDT_DD']==25+i-k) & (df['STT']==j)]['temp'])
        df[(df['SDT_DD']==25+i) & (df['STT']==j)]['temp']= newTemp / 5
        df[(df['SDT_DD']==25+i) & (df['STT']==j)]['hum']= newHum / 5
        df[(df['SDT_DD']==25+i) & (df['STT']==j)]['dew']= newDew / 5
        df[(df['SDT_DD']==25+i) & (df['STT']==j)]['windSpeed']= newWindSpeed / 5
    
print(df)

     Unnamed: 0  SDT_YY  SDT_MM  SDT_DD  STT    ARP  temp  hum  dew  \
0           366    2019       9      16    6   ARP1    20   92   19   
1           366    2019       9      16    6   ARP1    20   92   19   
2           366    2019       9      16    6   ARP1    20   92   19   
3           366    2019       9      16    6   ARP1    20   92   19   
4           366    2019       9      16    6   ARP1    20   92   19   
5           366    2019       9      16    6   ARP1    20   92   19   
6           366    2019       9      16    6   ARP1    20   92   19   
7           366    2019       9      16    6   ARP1    20   92   19   
8           366    2019       9      16    6   ARP1    20   92   19   
9           366    2019       9      16    6   ARP1    20   92   19   
10          366    2019       9      16    6   ARP1    20   92   19   
11          366    2019       9      16    6   ARP1    20   92   19   
12          366    2019       9      16    6   ARP1    20   92   19   
13    

In [22]:
# YY랑 DD삭제
df.drop(columns=['SDT_YY', 'SDT_DD'], axis=1, inplace=True)

In [24]:
def one_hot_dummies(df, *args):
    for col in args:
        one_hot_col = pd.get_dummies(df[col])
        new_col = [col + '_' + str(s) for s in one_hot_col.columns]
        one_hot_col.columns = new_col
        df = df.drop([col], axis = 1)
        df = df.join(one_hot_col)

    return df
#'ARPODP' 일단 뺌
df = one_hot_dummies(df, 'SDT_MM', 'SDT_DY', 'ARP', 'ODP', 'FLO',  'STT')

In [25]:
df.columns


# 중요도 낮은 변수들 제거 

Index(['Unnamed: 0', 'temp', 'hum', 'dew', 'windSpeed', 'hpa', 'AOD',
       'SDT_MM_9', 'SDT_DY_월', 'SDT_DY_화', 'ARP_ARP1', 'ARP_ARP10',
       'ARP_ARP11', 'ARP_ARP12', 'ARP_ARP13', 'ARP_ARP14', 'ARP_ARP15',
       'ARP_ARP2', 'ARP_ARP3', 'ARP_ARP4', 'ARP_ARP5', 'ARP_ARP6', 'ARP_ARP7',
       'ARP_ARP8', 'ARP_ARP9', 'ODP_ARP1', 'ODP_ARP10', 'ODP_ARP11',
       'ODP_ARP12', 'ODP_ARP13', 'ODP_ARP14', 'ODP_ARP15', 'ODP_ARP2',
       'ODP_ARP3', 'ODP_ARP4', 'ODP_ARP5', 'ODP_ARP6', 'ODP_ARP7', 'ODP_ARP8',
       'ODP_ARP9', 'FLO_A', 'FLO_B', 'FLO_F', 'FLO_H', 'FLO_I', 'FLO_J',
       'FLO_L', 'FLO_M', 'STT_6', 'STT_7', 'STT_8', 'STT_9', 'STT_10',
       'STT_11', 'STT_12', 'STT_13', 'STT_14', 'STT_15', 'STT_16', 'STT_17',
       'STT_18', 'STT_19', 'STT_20', 'STT_21', 'STT_22'],
      dtype='object')

In [28]:
from sklearn.preprocessing import RobustScaler
import pickle
from sklearn.externals import joblib

def fogModel(df):
    # 날씨 missing 값들은 0으로 대체
    df.fillna(0, inplace = True)
    
    # 모델에서 쓰인 Scaling기법 적용
    scaler = RobustScaler()
    df[['hum', 'dew','temp','windSpeed']] = scaler.fit_transform(df[['hum', 'dew','temp','windSpeed']])
    
    # 저장된 모델 불러오기
    clf_from_joblib = joblib.load('fogmodel.pkl') 

    # 지연 율 저장
    fog_prob = clf_from_joblib.predict_proba(df)
    

    fog_column = []
    # dly_rate에 지연율 저장
    for i in fog_prob:
        fog_column.append(i[1])
        
    
    return fog_column

In [29]:
# df에서 fog관련 column만 함수에 넘김
fog = df[["temp","hum","dew","windSpeed"]]

fog_column = fogModel(fog)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [30]:
# 안개 column 추가
df['fog'] = fog_column

# 안개 관련 column 제거
df.drop(columns=['hum', 'dew','temp','windSpeed'], axis=1, inplace=True)
df.head()

,Unnamed: 0,hpa,AOD,SDT_MM_9,SDT_DY_월,SDT_DY_화,ARP_ARP1,ARP_ARP10,ARP_ARP11,ARP_ARP12,...,STT_14,STT_15,STT_16,STT_17,STT_18,STT_19,STT_20,STT_21,STT_22,fog
0,6,1016,D,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.815453
1,6,1016,D,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.815453
2,6,1016,D,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.815453
3,6,1016,D,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.815453
4,6,1016,D,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.815453


In [31]:
# 도착, 출발 데이터 분리
df_A = df[df['AOD']=='A']
df_D = df[df['AOD']=='D']

# AOD column삭제 
df_A = df_A.drop(['AOD'],axis = 1)
df_D = df_D.drop(['AOD'],axis = 1)

In [ ]:
# ------------------------------ #

In [ ]:
########## 아직 저장된 모델이 없음

# df_A 모델 -----------------------------------------------

# df_A 모델 불러오기
predict_dealy_A_joblib = joblib.load('predict_delay_A.pkl') 

# DLY 저장하기
dly_A = predict_dealy_A_joblib.predict(df_A)

# DLY_RATE 저장하기
dly_A_prob = predict_dealy_A_joblib.predict_proba(df_A)

dly_rate_A = []

# dly_rate에 지연율 저장
for i in dly_A_prob:
    dly_rate_A.append(i[1])
    
# DateFrame에 DLY, DLY_RATE추가
df.loc[dly['AOD']=='A','DLY'] = dly_A
df.loc[dly['AOD']=='A','DLY_RATE'] = dly_rate_A

# ---------------------------------------------------------

In [ ]:
# df_D 모델 -----------------------------------------------

# df_D 모델 불러오기
predict_dealy_D_joblib = joblib.load('predict_delay_D.pkl') 

# DLY 저장하기
dly_D = predict_dealy_D_joblib.predict(df_D)

# DLY_RATE 저장하기
dly_A_prob = predict_dealy_A_joblib.predict_proba(df_D)

dly_rate_D = []

# dly_rate에 지연율 저장
for i in dly_D_prob:
    dly_rate_D.append(i[1])
    
# DateFrame에 DLY, DLY_RATE추가
df.loc[dly['AOD']=='D','DLY'] = dly_D
df.loc[dly['AOD']=='D','DLY_RATE'] = dly_rate_D

# ---------------------------------------------------------

In [ ]:
# 라벨을 1,0에서 -> Y,N으로 
df.loc[dly['DLY']==1,'DLY'] = 'Y'
df.loc[dly['DLY']==0,'DLY'] = 'N'